<a href="https://colab.research.google.com/github/prtk2001/Face-Detection-Algorithm/blob/main/Clear_or_obstruction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
file_id = '1n2knrrDAShyoETHNIjz6LPjzxGJaKb3t'


# Download dataset
!gdown https://drive.google.com/uc?id={file_id}

# Unzip the downloaded file
!unzip -q clear-or-obstruction.zip

Downloading...
From: https://drive.google.com/uc?id=1n2knrrDAShyoETHNIjz6LPjzxGJaKb3t
To: /content/clear-or-obstruction.zip
73.0MB [00:02, 30.1MB/s]


In [2]:
file_id = '15o1lluSFLkU4WO3wucCkcAM9jphhwMMs'


# Download dataset
!gdown https://drive.google.com/uc?id={file_id}

# Unzip the downloaded file
!unzip -q validation-clear-or-obstruction.zip

Downloading...
From: https://drive.google.com/uc?id=15o1lluSFLkU4WO3wucCkcAM9jphhwMMs
To: /content/validation-clear-or-obstruction.zip
16.2MB [00:00, 26.4MB/s]


In [4]:
import os
import zipfile

# Directory with our training clear pictures
train_clear_dir = os.path.join('/content/clear-or-obstruction/clear')

# Directory with our training obstruction pictures
train_obstruction_dir = os.path.join('/content/clear-or-obstruction/obstruction')

# Directory with our training clear pictures
validation_clear_dir = os.path.join('/content/validation-clear-or-obstruction/clear')

# Directory with our training obstruction pictures
validation_obstruction_dir = os.path.join('/content/validation-clear-or-obstruction/obstruction')

train_clear_names = os.listdir(train_clear_dir)
print(train_clear_names[:10])

train_obstruction_names = os.listdir(train_obstruction_dir)
print(train_obstruction_names[:10])

validation_clear_names = os.listdir(validation_clear_dir)
print(validation_clear_names[:10])

validation_obstruction_names = os.listdir(validation_obstruction_dir)
print(validation_obstruction_names[:10])

['image_0069.jpg', 'image_0083.jpg', 'image_0115.jpg', 'image_0240.jpg', '0003_0000343_script_2.jpg', 'image_0084.jpg', 'image_0077.jpg', 'image_0073.jpg', 'image_0032.jpg', 'image_0021.jpg']
['00955_Mask.jpg', 'images (90).jpg', 'images - 2021-07-06T193059.967.jpg', 'images (46).jpg', 'download (30).jpg', 'images (47).jpg', 'images - 2021-07-06T193055.556.jpg', '00600_Mask.jpg', 'images (31).jpg', 'download (34).jpg']
['image_0426.jpg', 'image_0407.jpg', 'image_0447.jpg', 'image_0413.jpg', 'image_0410.jpg', 'image_0425.jpg', 'image_0440.jpg', 'image_0417.jpg', 'image_0434.jpg', 'image_0423.jpg']
['images (46).jpg', 'images (14).jpg', 'download (16).jpg', 'images - 2021-07-06T193105.012.jpg', 'images (21).jpg', '00961_Mask.jpg', 'images (44).jpg', 'images (86).jpg', 'images (61).jpg', 'images (10).jpg']


In [5]:
print('total training clear images:', len(os.listdir(train_clear_dir)))
print('total training obstruction images:', len(os.listdir(train_obstruction_dir)))
print('total validation clear images:', len(os.listdir(validation_clear_dir)))
print('total validation obstruction images:', len(os.listdir(validation_obstruction_dir)))

total training clear images: 270
total training obstruction images: 237
total validation clear images: 60
total validation obstruction images: 60


In [7]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        3

In [8]:
ACCURACY_THRESHOLD = 0.98
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('acc') > ACCURACY_THRESHOLD):   
          print("\nReached 98 accuracy, so stopping training!!" )   
          self.model.stop_training = True

callbacks = myCallback()

In [11]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['acc'])

from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/content/clear-or-obstruction/',  
        target_size=(300, 300),  
        batch_size=64,
        class_mode='binary')
validation_generator = validation_datagen.flow_from_directory(
        '/content/validation-clear-or-obstruction/',  
        target_size=(300, 300),  
        batch_size=8,
        
        class_mode='binary')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=8,  
      epochs=8,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8,
      callbacks=[callbacks])

Found 507 images belonging to 2 classes.
Found 120 images belonging to 2 classes.


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/8
8/8 [==============================] - 47s 6s/step - loss: 0.7092 - acc: 0.5720 - val_loss: 0.5318 - val_acc: 0.6875
Epoch 2/8
8/8 [==============================] - 45s 6s/step - loss: 0.5457 - acc: 0.6982 - val_loss: 0.4470 - val_acc: 0.7188
Epoch 3/8
8/8 [==============================] - 45s 6s/step - loss: 0.4441 - acc: 0.7613 - val_loss: 0.6489 - val_acc: 0.6250
Epoch 4/8
8/8 [==============================] - 45s 6s/step - loss: 0.4671 - acc: 0.7988 - val_loss: 0.2809 - val_acc: 0.9062
Epoch 5/8
8/8 [==============================] - 45s 6s/step - loss: 0.3944 - acc: 0.8383 - val_loss: 0.3185 - val_acc: 0.7969
Epoch 6/8
8/8 [==============================] - 45s 6s/step - loss: 0.2794 - acc: 0.8915 - val_loss: 0.7793 - val_acc: 0.7344
Epoch 7/8
8/8 [==============================] - 45s 6s/step - loss: 0.2778 - acc: 0.8797 - val_loss: 0.2019 - val_acc: 0.9219
Epoch 8/8
8/8 [==============================] - 45s 6s/step - loss: 0.2672 - acc: 0.9231 - val_loss: 0.1892 - 

In [12]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a clear")
  else:
    print(fn + " is a obstructed")

Saving sheetal.jpeg to sheetal.jpeg
Saving Screenshot_20210311-165320.png to Screenshot_20210311-165320.png
Saving IMG-20210420-WA0008.jpg to IMG-20210420-WA0008.jpg
Saving jyoti4.jpg to jyoti4.jpg
Saving IMG20190913174236_2.jpg to IMG20190913174236_2.jpg
[1.]
sheetal.jpeg is a clear
[1.]
Screenshot_20210311-165320.png is a clear
[0.]
IMG-20210420-WA0008.jpg is a obstructed
[0.]
jyoti4.jpg is a obstructed
[2.4533266e-24]
IMG20190913174236_2.jpg is a obstructed
